# 文生图-Dreambooth


https://github.com/open-mmlab/mmagic/tree/main/configs

## 训练之前的推理

In [ ]:
from mmengine import Config

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules
from mmengine.registry import init_default_scope 

init_default_scope('mmagic')
register_all_modules()

In [ ]:
cfg = Config.fromfile('configs/dreambooth-lora.py')

In [ ]:
dreambooth_lora = MODELS.build(cfg.model)
dreambooth_lora = dreambooth_lora.cuda()

In [ ]:
samples = dreambooth_lora.infer(prompt='side view of sks dog', guidance_scale=5)

In [ ]:
samples['samples'][0]

## 在数据集上训练Dreambooth

### 查看训练图片

训练数据集可从 https://drive.google.com/drive/folders/1BO_dyz-p65qhBRRMRA4TbZ8qW4rB99JZ 下载得到，并放到 data/ 目录下 

In [ ]:
from PIL import Image 

Image.open('data/dog6/imgs/00.jpg')

In [ ]:
# !bash tools/dist_train.sh configs/dreambooth/dreambooth.py 1
    
!python -m torch.distributed.launch --nproc_per_node=1 train.py configs/dreambooth-lora.py --launcher pytorch

## 用训练好的模型做预测

In [ ]:
import torch 
state = torch.load('data/dreambooth_dog1000.pth')['state_dict']

In [ ]:
def convert_state_dict(state):
    state_dict_new = {}
    for k, v in state.items():
        if '.module' in k:
            k_new = k.replace('.module', '')
        else:
            k_new = k
        state_dict_new[k_new] = v
    return state_dict_new

In [ ]:
dreambooth_lora.load_state_dict(convert_state_dict(state), strict=False)
dreambooth_lora = dreambooth_lora.cuda()

In [ ]:
samples = dreambooth_lora.infer('side view of sks dog', guidance_scale=5)

In [ ]:
samples['samples'][0]

In [ ]:
samples = dreambooth_lora.infer('ear close-up of sks dog', guidance_scale=5)

In [ ]:
samples['samples'][0]